In [ ]:
import os
import zipfile
import shutil
import csv

# Ruta del archivo ZIP y la carpeta de salida
zip_path = 'C:/Users/esteban.correa/Downloads/pdfs.zip'
output_folder = 'C:/Users/esteban.correa/Downloads/SAMPLING_ORDENADO'

# Archivo CSV con la información de Ciudad, Producto y Tienda
csv_path = "C:/Users/esteban.correa/Downloads/Automatizacion_Sampling_2025_03_12.csv"

# Paso 1: Descomprimir la carpeta ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_folder)

# Paso 2: Leer el archivo CSV para obtener la distribución Ciudad/Producto/Tienda
orders_info = {}
with open(csv_path, mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        orders_info[row['order_id']] = row

# Paso 3: Reorganizar los archivos según la distribución
for root, dirs, files in os.walk(output_folder):
    for file in files:
        if file.startswith('PO-'):
            order_id = file[3:].split('.')[0]  # Extrae el order_id del nombre del archivo
            if order_id in orders_info:
                info = orders_info[order_id]
                city = info['city']
                product = info['product_name']
                store = info['wh_name']
                
                # Crear la nueva estructura de carpetas
                new_dir = os.path.join(output_folder, city, product)
                os.makedirs(new_dir, exist_ok=True)
                
                # Mover el archivo a la nueva ubicación
                old_path = os.path.join(root, file)
                new_path = os.path.join(new_dir, store + os.path.splitext(file)[1])  # Cambia el nombre del archivo por el de la tienda
                shutil.move(old_path, new_path)

# Paso 4: Eliminar archivos PO- del directorio raíz
for file in os.listdir(output_folder):
    file_path = os.path.join(output_folder, file)
    if os.path.isfile(file_path) and file.startswith('PO-'):
        os.remove(file_path)